In [1]:
print("OK")

OK


In [2]:
%pwd

'd:\\Medical-chatbot-End-to-End-Generative-Ai\\research'

In [3]:
%pwd

'd:\\Medical-chatbot-End-to-End-Generative-Ai\\research'

In [4]:
import os
os.chdir("../")

In [5]:
#pwd

In [6]:
%pwd

'd:\\Medical-chatbot-End-to-End-Generative-Ai'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [9]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [12]:
#text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\HP\AppData\Local\Temp\ipykernel_2532\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\HP\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
print(embeddings)


client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
#query_result

In [19]:
import os
from dotenv import load_dotenv

load_dotenv()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')

if pinecone_api_key:
    print("Pinecone API Key loaded successfully.")
else:
    print("Pinecone API Key not found. Check your .env file.")


Pinecone API Key loaded successfully.


In [20]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip uninstall pinecone-plugin-inference -y


Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install --upgrade pinecone


In [23]:

from dotenv import load_dotenv
load_dotenv()

True

In [24]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [25]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'f4f5f727687044f45770cd4015007a19', 'Date': 'Fri, 28 Feb 2025 12:39:51 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [26]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

# Delete the existing index
pc.delete_index("medicalbot")

# Recreate the index
pc.create_index(
    name="medicalbot",
    dimension=384, 
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

print("Index reset successfully! 🎯")


Index reset successfully! 🎯


In [27]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [28]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [29]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [30]:
docsearch

In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [32]:
retrieved_docs = retriever.invoke("What is Acne?")

In [33]:
retrieved_docs

[Document(id='6d2f9281-89ac-4e5b-a9b9-b9fd721c661f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='2c603eee-6098-4fb3-9bd4-472926d2d6a9', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ

In [34]:
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i+1} (Page {doc.metadata['page_label']}):")
    print(doc.page_content[:500])  # Display first 500 characters for readability



Document 1 (Page 40):
GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26

Document 2 (Page 39):
GALE ENCYCLOPEDIA OF MEDICINE 2 25
Acne
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceous
glands become inflamed.(Photograph by Biophoto Associ-
ates, Photo Researchers, Inc. Reproduced by permission.)
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25

Document 3 (Page 38):
Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when the
pores of the skin become clogged with oil, dead skin
cells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is
the most common skin disease. It affects nearly 17 million
people in the United States. While acne can arise at any


In [35]:
pip install google-generativeai


Note: you may need to restart the kernel to use updated packages.


In [44]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")


In [46]:
import os

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    print("Error: GEMINI_API_KEY is not set!")
else:
    print("API Key Found!")


API Key Found!


In [47]:
pip install --upgrade google-generativeai


Note: you may need to restart the kernel to use updated packages.


In [56]:
import google.generativeai as genai

# Configure API Key
genai.configure(api_key=api_key)

# List available models
models = genai.list_models()
for model in models:
    print(model.name)


models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experim

In [57]:
import google.generativeai as genai

# Configure API Key
genai.configure(api_key=api_key)

# Load the correct model
model = genai.GenerativeModel("gemini-1.5-pro-latest")  # Change model name if needed

# Generate a response
response = model.generate_content("What is Acne?")
print(response.text)


Acne is a common skin condition that occurs when hair follicles become clogged with oil, dead skin cells, and sometimes bacteria.  This can lead to a variety of blemishes, including:

* **Whiteheads:** Closed, plugged pores.
* **Blackheads:** Open, plugged pores. The dark color is not dirt, but rather oxidized oil.
* **Papules:** Small, red, raised bumps.
* **Pustules:** Similar to papules, but with a white or yellow pus-filled tip.
* **Nodules:** Large, solid, painful lumps beneath the skin.
* **Cysts:** Painful, pus-filled lumps beneath the skin. These can cause scarring.

Several factors contribute to acne development:

* **Excess oil production (sebum):**  Hormonal changes, particularly during puberty, can increase sebum production.
* **Clogged hair follicles:**  Dead skin cells and excess oil can clog pores.
* **Bacteria:** *Propionibacterium acnes* (P. acnes), a bacterium normally present on the skin, can thrive in clogged pores and contribute to inflammation.
* **Inflammation:**

In [58]:
import google.generativeai as genai
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Configure Gemini API Key
genai.configure(api_key=api_key)

# Define Gemini LLM
model = genai.GenerativeModel("gemini-1.5-pro-latest")

# Define System Prompt
system_prompt = (
    "You are an AI assistant for medical question-answering tasks. "
    "Use the retrieved context to provide accurate answers. "
    "If you don't know the answer, state that clearly. "
    "Keep the response within three sentences."
    "\n\n"
    "{context}"
)

# Define Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Define the Question Answering Chain
def query_gemini(context, input_text):
    """Generate response using Gemini with provided context."""
    full_prompt = system_prompt.format(context=context) + "\n" + input_text
    response = model.generate_content(full_prompt)
    return response.text

# Retrieval-Augmented Generation (RAG) Chain
def rag_chain(retriever, input_text):
    """Retrieve documents and generate answers using Gemini."""
    retrieved_docs = retriever.get_relevant_documents(input_text)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    return query_gemini(context, input_text)




In [59]:

response1 = rag_chain(retriever, "What is Acromegaly and gigantism?")
print(response1)



Acromegaly and gigantism are disorders caused by abnormal growth hormone release from the pituitary gland, leading to increased bone and soft tissue growth.  Gigantism occurs when this abnormal growth hormone release happens before bone growth stops, resulting in unusual height, while acromegaly occurs after bone growth has stopped. Acromegaly affects approximately 50 out of every one million people and often goes undiagnosed until middle age due to its gradual onset of symptoms.



In [60]:
response2 = rag_chain(retriever, "What is stats?")
print(response2)

This text describes a Complete Blood Count (CBC), a common blood test evaluating red blood cells, white blood cells, and platelets.  It doesn't contain any information about "stats."  Therefore, I cannot answer your question using the provided context.



In [61]:
response3 = rag_chain(retriever, "What is Abdominal wall defects?")
print(response3)

Abdominal wall defects are birth defects that cause the stomach or intestines to protrude.  This occurs because of an opening in the abdominal wall.  The defects are usually surgically repairable.



In [62]:
response4 = rag_chain(retriever, "What is Acute gout attack?")
print(response4)

An acute gout attack is characterized by sudden, severe pain, redness, and swelling in a joint, most commonly the big toe.  These attacks are triggered by high levels of uric acid, forming crystals in the joint.  While long-term gout treatment aims to lower uric acid, these medications are avoided during acute attacks as they can worsen symptoms; instead, non-steroidal anti-inflammatory drugs are used for pain relief.



In [63]:
response4 = rag_chain(retriever, "Remedy for Acute gout attack?")
print(response4)

For acute gout attacks, non-steroidal anti-inflammatory drugs (like indomethacin), colchicine, and corticosteroids are used.  The effectiveness of treatment depends on how quickly medication is administered after the attack begins.  Lowering uric acid levels during an acute attack can worsen symptoms, so those medications are used for long-term management.



In [64]:
response4 = rag_chain(retriever, "medication for Acute gout attack?")
print(response4)

Medications used for acute gout attacks include non-steroidal anti-inflammatory drugs (such as indomethacin), colchicine, and corticosteroids.  The most important factor in effective treatment is how quickly medication is administered after the attack begins.  Lowering uric acid levels during an acute attack can worsen symptoms, so drugs targeting long-term uric acid management are avoided in the short term.

